In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

!ls

Mounted at /content/gdrive/
gdrive	sample_data


In [2]:
import os

home_dir = '/content/gdrive/My Drive/AYield/China'
os.chdir(home_dir)
!pwd
# !ls
# !pip3 install --upgrade IPython
!pip install fire
!pip install geopandas
!pip install geojson
!pip install tqdm

from tqdm import tqdm
import geojson

import geopandas as gpd

from glob import glob

/content/gdrive/My Drive/AYield/China
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 3.3 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=8c0a5e7b9eb9ef05ac59d9e48fdb15b36bcdefa9818353d7f45d90331c360fc9
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.8 MB/s 
     |████████████████████████████████| 16.7 MB 403 kB/s 
     |████████████████████████████████| 6.3 MB 42.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
code_dir = os.path.join(home_dir, 'pycrop-yield-prediction')
os.chdir(code_dir)
!pwd
!ls

/content/gdrive/My Drive/AYield/China/pycrop-yield-prediction
cyp  data  diagrams  environment.yml  LICENSE  README.md  run.py


In [4]:
from cyp.data.utils import load_clean_yield_data as load
import ee
import ssl
import time
from pathlib import Path
import numpy as np
import shapely
from shapely import wkt
import os

locations_filepath=Path("data/china_yield/wheat_2002-2018.csv")
locations = load(locations_filepath)

In [5]:
# fn
!earthengine authenticate
!earthengine task cancel all

ee.Initialize()

!pwd
!ls

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=uNPkx0E4-mgJ-853W0uJ8f5TllpILlCmWrzTBQTzIug&tc=2o3prsC5sqo__s7-8RfDAJu2IDUfq-EU8Muo8Q7d1Uo&cc=C2MwZvtxk7TApbe6_xBhWRH7nN07FJFyHDC2AN_Q3IE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qgOuDioXzgZxk1mJLCMjfHdnL_uu9ZSmAF0brGlVhfOfmr4zTOmz5s

Successfully saved authorization token.
/content/gdrive/MyDrive/AYield/China/pycrop-yield-prediction
cyp  data  diagrams  environment.yml  LICENSE  README.md  run.py


In [18]:
from shapely import geometry
import geopandas as gpd
import pandas as pd

########################################################################
df = pd.DataFrame(columns=["CntyFips", "StateFips", "Longitude", "Latitude"])
features=[]
memCounty = '-1'
icount = 0
for index, row in locations.iterrows():
    # polyg = i.geometry
    # print(type(row), row['geometry'])
    if str(row['County ANSI']) == memCounty:
      continue
    else:
      polyg = wkt.loads(row['geometry'])
      # print('polyg : ', index, type(polyg), polyg)
      polyg_gson = [{'type': 'Feature', 'properties': {}, 'geometry': shapely.geometry.mapping(polyg)}]
      # print(type(polyg_gson[0]['geometry']['coordinates']), list(polyg_gson[0]['geometry']['coordinates'][0]))
      tuples = list(polyg_gson[0]['geometry']['coordinates'][0])
      polyg_lists = [list(x) for x in tuples]
      # print('append :', 'County ANSI ', str(row['County ANSI']), 'State ANSI ', str(row['State ANSI']), type(polyg_lists), polyg_lists)
      g = ee.Geometry.Polygon(polyg_lists) 
      # print(type(g), g.getInfo())
      # print(g['coordinates'])
      polyg = geometry.Polygon(g['coordinates'][0])
      Long = polyg.centroid.wkt.split('(')[-1].split(')')[0].split(' ')[0]
      Latd = polyg.centroid.wkt.split('(')[-1].split(')')[0].split(' ')[1]
      print( 'centroid : ', index, Long, Latd )
      df.loc[icount] = [ str(row['County ANSI']) ] + [ str(row['State ANSI']) ] + [Long] + [Latd]
      icount = icount + 1
      # df.append( pd.DataFrame([{'CntyFips':str(row['County ANSI']), 'StateFips':str(row['State ANSI']), 'Longitude':Long, 'Latitude':Latd}]) )    
      # dff = pd.DataFrame([{'CntyFips':str(row['County ANSI']), 'StateFips':str(row['State ANSI']), 'Longitude':Long, 'Latitude':Latd}]) 
      # dff.head()
      # fn
      # feature = ee.Feature(g, {'County ANSI':ee.String(str(row['County ANSI'])), 'State ANSI':ee.String(str(row['State ANSI']))})
      # features.append(feature)
      # print('Feature appended : ', type(feature)) #, feature.getInfo())
      # print(len(polyg_lists), feature['geometry']['coordinates'])
      # fn
    memCounty = str(row['County ANSI'])
# df = pd.DataFrame([df])
df.to_csv('county_data.csv')
print("Feature collection done")               
# region = ee.FeatureCollection(features)
########################################################################


centroid :  1 116.40972275437531 40.18415999235992
centroid :  17 117.32987574478915 39.31286148003779
centroid :  34 116.11955586981836 39.544570980275545
centroid :  51 112.29325534733441 37.57493902706648
centroid :  68 113.91213472113701 44.086704308508686
centroid :  85 122.60594488913884 41.30356410246778
centroid :  102 126.19149158689369 43.665606785715646
centroid :  119 127.7567091055589 47.85352141344163
centroid :  136 121.46775432162043 31.279137356847652
centroid :  153 119.44429502403912 32.97476653518012
centroid :  170 121.13558578019368 27.80361096191242
centroid :  187 117.22595580205993 31.825810603647902
centroid :  204 117.9740856119924 26.088186817122335
centroid :  221 115.72064428213676 27.613727022063458
centroid :  238 118.13661574255075 36.332705276118446
centroid :  255 113.61470277231409 33.88249074396553
centroid :  272 112.26947404594185 30.975562585666392
centroid :  289 111.70802733133482 27.607109198844054
centroid :  323 108.7858425899112 23.83037670